In [144]:
import math
import random
import gzip
import struct
import copy

In [145]:
class Layer():
    def __init__(self, num_of_units, activation="relu"):
        self._a_in = None 
        self.num_of_units = num_of_units
        self.activation=activation
        self.biases = [0.0 for _ in range(num_of_units)]
        self.weights = None
    @property
    def a_in(self):
        return self._a_in
    @a_in.setter
    def a_in(self, value):
        self._a_in = value
        if self.weights is None:
            fan_in = len(self._a_in)
            std = math.sqrt(2.0 / fan_in)  # He initialization
            self.weights = [[random.gauss(0, std) for _ in range(fan_in)] 
                           for _ in range(self.num_of_units)]
    def infer(self):
        a_out = [0]*self.num_of_units
        for i_unit in range(self.num_of_units):
            z = self.biases[i_unit]
            for i_feature in range(len(self._a_in)):
                z += self._a_in[i_feature] * self.weights[i_unit][i_feature]
            if self.activation=="relu":
                a_out[i_unit] = 0 if z < 0 else z #ReLU
            elif self.activation=="linear":
                a_out[i_unit] = z
        return a_out

In [164]:
class MultiClassificationNet():
    def __init__(self, *layers):
        self.layers = layers
        self.best_weights_found = []
        self.best_biases_found = []
        # Activation values cache for faster backprop
        self.activation_values = None

    def infer(self, initial_a_in):
        activation_value = initial_a_in
        self.activation_values = []
        self.activation_values.append(initial_a_in)
        for layer in self.layers:
            layer.a_in = activation_value
            activation_value = layer.infer()
            self.activation_values.append(activation_value)
        # Softmaxing
        max_z = max(activation_value) # to avoid overflow
        exponents = [math.exp(z - max_z) for z in activation_value]
        exponents_sum = sum(exponents)
        return [exp / exponents_sum for exp in exponents]
    
    def train(self, X, y, epochs):
        best_epoch_loss = 1
        for epoch in range(epochs):
            total_loss = 0
            for i, x in enumerate(X):
                y_hat = self.infer(x) # probabilities
                correct_y_hat = sum([y_hat[j] * y[i][j] for j in range(len(y_hat))])
                total_loss += -1 * math.log(correct_y_hat)
    
                # Initializing lists
                d_weights = []
                d_biases = []
                for layer in self.layers:
                    d_weights.append([[0]*len(layer.a_in) for _ in range(layer.num_of_units)])
                    d_biases.append([0]*layer.num_of_units)
                    
                ## Backpropagaaatioooooon!
                dL_dz = [y_hat[j] - y[i][j] for j in range(len(y_hat))]
                da_carried = dL_dz
                for rev_i, layer in enumerate(reversed(self.layers)):
                    layer_i = len(self.layers) - 1 - rev_i
    
                    # ReLU's impact on derivatives ----TODO-----
                    a_out = self.activation_values[layer_i+1]
                    for neuron_i in range(layer.num_of_units):
                        if layer.activation == "relu":
                            da_carried[neuron_i] *= 1 if a_out[neuron_i] > 0 else 0
                            
                    # Partial derivatives of weights and biases
                    for neuron_i in range(layer.num_of_units):
                        d_biases[layer_i][neuron_i] = da_carried[neuron_i]
                        for weight_i in range(len(layer.a_in)):
                            d_weights[layer_i][neuron_i][weight_i] = (
                                da_carried[neuron_i] * layer.a_in[weight_i]
                            )
                    # Partial derivatives of the previous layer's activations
                    new_da_carried = [0]*len(layer.a_in)
                    for prev_a_i in range(len(layer.a_in)):
                        for neuron_i in range(layer.num_of_units):
                            new_da_carried[prev_a_i] += da_carried[neuron_i] * layer.weights[neuron_i][prev_a_i]

                    da_carried = new_da_carried
    
    
                
                # Gradient Descent
                learning_rate = 0.005
                for layer_i, layer in enumerate(self.layers):
                    for neuron_i in range(layer.num_of_units):
                        layer.biases[neuron_i] -= learning_rate * d_biases[layer_i][neuron_i]
                        for weight_i in range(len(layer.a_in)):
                            layer.weights[neuron_i][weight_i] -= learning_rate * d_weights[layer_i][neuron_i][weight_i]

            # Printing epoch loss and saving best parameters
            epoch_loss = total_loss/len(X)
            print(f"Epoch {epoch}. Loss: {epoch_loss}")
            if epoch_loss < best_epoch_loss:
                self.best_weights_found = []
                self.best_biases_found = []
                for layer in self.layers:
                    layer_weights = []
                    for neuron_i in range(layer.num_of_units):
                        neuron_weights = [layer.weights[neuron_i][weight_i] for weight_i in range(len(layer.a_in))]
                        layer_weights.append(neuron_weights)
                    self.best_weights_found.append(layer_weights)
                    self.best_biases_found.append(layer.biases[:])  # Deep copy biases
                best_epoch_loss = epoch_loss
                save_best_weights(self, "best_params.json")

    def evaluate(self, X_test, y_test):
        correct = 0
        for x, y in zip(X_test, y_test):
            y_hat = self.infer(x)
            predicted = y_hat.index(max(y_hat))
            true_label = y.index(max(y))
            if predicted == true_label:
                correct += 1
        accuracy = correct / len(X_test)
        return accuracy

In [165]:
# Load MNIST - Blatantly copied from Grok - no questions asked
def load_mnist(subset_size=10000):
    def read_images(filename):
        with gzip.open(filename, 'rb') as f:
            magic, num, rows, cols = struct.unpack('>IIII', f.read(16))
            images = []
            for _ in range(num):
                pixels = [b / 255.0 for b in f.read(rows * cols)]  # Normalize to [0, 1]
                images.append(pixels)
        return images

    def read_labels(filename):
        with gzip.open(filename, 'rb') as f:
            magic, num = struct.unpack('>II', f.read(8))
            labels = [b for b in f.read(num)]
            # Convert to one-hot encoding
            one_hot = [[1 if j == label else 0 for j in range(10)] for label in labels]
        return one_hot

    X_train = read_images('Documents/Datasets/MNIST/train-images-idx3-ubyte.gz')[:subset_size]
    y_train = read_labels('Documents/Datasets/MNIST/train-labels-idx1-ubyte.gz')[:subset_size]
    X_test = read_images('Documents/Datasets/MNIST/t10k-images-idx3-ubyte.gz')
    y_test = read_labels('Documents/Datasets/MNIST/t10k-labels-idx1-ubyte.gz')
    return X_train, y_train, X_test, y_test

In [166]:
# Predict image
from PIL import Image
from IPython.display import display

def predict_image(jpg_filename, neural_network, display_image=True):
    img = Image.open(jpg_filename)
    img = img.convert('L')  # Convert to grayscale
    img = img.resize((28, 28))  # Resize to 28x28 if necessary
    
    pixels = [pixel / 255.0 for pixel in img.getdata()]
    
    prediction = neural_network.infer(pixels)
    
    predicted_digit = prediction.index(max(prediction))
    
    if display_image:
        display(img)
    
    return predicted_digit

In [ ]:
# Run experiment
X_train, y_train, X_test, y_test = load_mnist(subset_size=10000)
neural_network = MultiClassificationNet(Layer(16, "relu"), Layer(16, "relu"), Layer(10, "linear"))
neural_network.train(X_train, y_train, epochs=100)
accuracy = neural_network.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Epoch 0. Loss: 0.5471447907119127
Epoch 1. Loss: 0.2971334920255449
Epoch 2. Loss: 0.23862759410115306
Epoch 3. Loss: 0.20143824293538612
Epoch 4. Loss: 0.1738633263816597
Epoch 5. Loss: 0.1556997583326201


In [ ]:
import json

def save_best_params(neural_network, filename):
    model_data = {
        'best_weights_found': neural_network.best_weights_found,
        'best_biases_found': neural_network.best_biases_found
    }
    with open(filename, 'w') as f:
        json.dump(model_data, f, indent=4)

def restore_best_params(neural_network):
        if neural_network.best_weights_found is None or neural_network.best_biases_found is None:
            raise ValueError("No best parameters found.")
        for layer, weights, biases in zip(neural_network.layers, neural_network.best_weights_found, neural_network.best_biases_found):
            layer.weights = copy.deepcopy(weights)
            layer.biases = copy.deepcopy(biases)

def load_best_params(neural_network, filename):
    with open(filename, 'r') as f:
        model_data = json.load(f)
    neural_network.best_weights_found = model_data['best_weights_found']
    neural_network.best_biases_found = model_data['best_biases_found']
    # Restore weights and biases to the layers
    restore_best_params(neural_network)

In [162]:
# save_best_weights(neural_network, "best_params.json")
load_best_params(neural_network, "best_params.json")
predict_image("Pictures/canvas.png", neural_network)

0

In [ ]:
'''

'''